# Imports

In [16]:
import json, time, os
from tqdm.notebook import tqdm_notebook

with open('/Users/aveld/secret/tmdb_api.json') as f:
    login = json.load(f)

In [17]:
import tmdbsimple as tmdb

tmdb.API_KEY = login['api-key']

In [18]:
import pandas as pd
import numpy as np

In [19]:
# Function to get MPAA
def movie_rating(id):
    """Adapted from source = https://github.com/celiao/tmdbsimple"""
    
    # Get the movie object for the current id
    movie = tmdb.Movies(id)
    # save the .info .releases dictionaries
    info = movie.info()
    releases = movie.releases()
    # Loop through countries in releases
    for c in releases['countries']:
        # if the country abbreviation==US
        if c['iso_3166_1' ] =='US':
            ## save a "certification" key in the info dict with the certification
            info['certification'] = c['certification']
    return info

In [20]:
# Function to add new data to a JSON file
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [21]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete JSON file
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)  

In [22]:
basics = pd.read_csv('movie-data/title_basics.csv.gz')

In [23]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,118.0,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,70.0,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,122.0,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,100.0,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,126.0,Drama


In [24]:
# Lists
YEARS = [2019, 2020, 2021]
errors = []

In [31]:
# Nested Loop for movies post pandemic
for i in tqdm_notebook(YEARS, desc='YEARS', position=0):
    JSON_FILE = f"movie-data/tmdb_api_results_{i}.json"
    create_json_file(JSON_FILE, delete_if_exists=True)
    
    #Saving new year as the current df
    df = basics.loc[basics['startYear']==i].copy()
    # saving movie ids to list
    movie_ids = df['tconst'].copy()
    
    # Keeping track of previous loops
    previous_df = pd.read_json(JSON_FILE)
    
    # Making sure not to repeat movie IDs
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {i}',
                                  position=1,
                                  leave=True):
        try:
            # Retrieve data for the movie id
            temp = movie_rating(movie_id)  
            # Append/extend results to existing file
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep
            time.sleep(0.02)
            
        except Exception as e:
            errors.append([movie_id, e])
    
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"movie-data/final_tmdb_data_{i}.csv.gz",
                         compression="gzip", index=False)`


print(f"- Total errors: {len(errors)}")

YEARS:   0%|          | 0/4 [00:00<?, ?it/s]

[i] movie-data/tmdb_api_results_2019.json not found. Saving empty list to new file.


Movies from 2019:   0%|          | 0/5765 [00:00<?, ?it/s]

[i] movie-data/tmdb_api_results_2020.json not found. Saving empty list to new file.


Movies from 2020:   0%|          | 0/4913 [00:00<?, ?it/s]

[i] movie-data/tmdb_api_results_2021.json not found. Saving empty list to new file.


Movies from 2021:   0%|          | 0/4985 [00:00<?, ?it/s]

[i] movie-data/tmdb_api_results_2022.json not found. Saving empty list to new file.


Movies from 2022: 0it [00:00, ?it/s]

- Total errors: 6292


I was only able to retrieve info on the movies in the basics dataframe up until 2022.

In [33]:
df_2019 = pd.read_csv('movie-data/final_tmdb_data_2019.csv.gz')
df_2020 = pd.read_csv('movie-data/final_tmdb_data_2020.csv.gz')
df_2021 = pd.read_csv('movie-data/final_tmdb_data_2021.csv.gz')

In [36]:
old_info = pd.read_csv('movie-data/tmdb_results_combined.csv.gz')

In [37]:
new_info = pd.concat([df_2019, df_2020, df_2021, old_info])

In [38]:
new_info.to_csv('movie-data/tmdb_results_combined_final.csv.gz', compression='gzip', index=False)

In [46]:
df = new_info[['imdb_id', 'revenue', 'budget', 'certification']].copy()

In [49]:
df = df.dropna()

In [51]:
df = df.reset_index()

In [52]:
df

,index,imdb_id,revenue,budget,certification
0,1,tt0385887,18377736.0,26000000.0,R
1,2,tt0437086,404980000.0,170000000.0,PG-13
2,3,tt0441881,2078370.0,23934823.0,R
3,4,tt0448115,366080049.0,80000000.0,PG-13
4,5,tt0783640,3364426.0,20000000.0,R
...,...,...,...,...,...
2713,595,tt0293416,4035192.0,15000000.0,PG-13
2714,612,tt0311361,0.0,100000.0,R
2715,614,tt0322645,0.0,10540.0,PG-13
2716,615,tt0347014,0.0,1200000.0,NR
